In [2]:
from inventree.api import InvenTreeAPI
from inventree import company  # noqa: E402
from inventree.stock import StockItem, StockLocation  # noqa: E402
from inventree.company import SupplierPart
from inventree.part import Part, PartAttachment, PartCategory, PartCategoryParameterTemplate, Parameter, ParameterTemplate, PartTestTemplate, PartRelated, BomItem  # noqa:E402

In [3]:
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
import sys
import getpass
from tqdm import tqdm

In [4]:
#登录Inventree
def login_inventree(url,username,password):
    try:
        api = InvenTreeAPI(host=url, username=username, password=password)
        print('登录成功！')
        return api
    except:
        #print('登录失败！')
        raise(ValueError('登录失败'))

In [5]:
#获取立创商城的订单（手动）保存到txt中
#手动获取订单信息的方法
def read_lcsc_order(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            soup= BeautifulSoup(f.read(), 'html.parser')
            print('文件读取成功！')
            return soup
    except:
        #print('文件读取失败.....')
        raise(ValueError('文件读取失败'))

In [6]:
#处理订单数据
#获取元器件名称 编号 链接 封装 etc
#获取元器件图片链接
def deal_order_data(soup):
    df=pd.DataFrame(columns=['IPN','name','descripition','keyword','footprint','link','img','stock','price'])
    try:
        divs=soup.find_all(class_='order-a-item')
        for i in divs:
            wpercent=i.find_all(class_='w-percent-15')
            dict = {'IPN':i.img['alt'],
                   'link':i.a['href'],
                    'img':i.img['src'],
                    'name':i.li.text.rstrip().split('/')[0].rstrip(),
                    'descripition':i.li.text.rstrip().split('/')[1].lstrip(),
                    'footprint':i.a['title'].split('/')[1].lstrip(),
                    'keyword':i.a.text.strip(),
                    'stock':wpercent[0].text[:-1],
                    'price':wpercent[2].text[1:-2]
                   }
            data = pd.DataFrame(dict,index=[divs.index(i)])
            df=pd.concat([df,data])
            #print(i.img['alt'],i.img['src'],i.img['title'],i.li.text.rstrip(),i.a['href'])
            #print(i.a['title'],i.a.text.strip(),wpercent[0].text[:-1],wpercent[2].text[1:-2])
            download_lcsc_pics(i.img['src'],r'lc/'+i.img['alt']+'.jpg')
        return df
    except:
        print('文件解析失败....')

In [7]:
#下载图片
#便于LCSC的图片上传到Inventree中
#图片命名唯一和IPN一致
def download_lcsc_pics(img_link,img_path):
    urllib.request.urlretrieve(img_link,img_path)

In [8]:
#保存数据到excel中（可选）
def save_datas_to_excel():
    return 0

In [9]:
def Inventree_read_PartCategory(api):
    cats = PartCategory.list(api)
    print('pk ','name')
    for i in cats:
        print(i.pk,'  ',i.name);
    cat_pk=input('请输入插入的分类pk号：')
    return cat_pk

In [10]:
def Inventree_read_company(api):
    coms=company.Company.list(api)
    print('pk ','name')
    for i in coms:
        print(i.pk,'  ',i.name);
    company_pk=input('请输入插入的供应商pk号：')
    return company_pk

In [11]:
def Inventree_read_location(api):
    locs = StockLocation.list(api)
    print('pk ','name')
    for i in locs:
        print(i.pk,'  ',i.name);
    loc_pk=input('请输入插入的默认仓库pk号：')
    return loc_pk

In [12]:
def Inventree_Part_create(api,df,loc_pk,company_pk,cat_pk):
    pbar = tqdm(len(df.index))
    for i in range(0,len(df.index)):
        try:
            couch = Part.create(api, {
                'name': df.loc[i]['name'],
                'description': df.loc[i]['descripition'],
                'category': cat_pk,
                'active': True,
                'virtual': False,
                'IPN':df.loc[i]['IPN'],
                'link':df.loc[i]['link'],
                'keywords':df.loc[i]['keyword'],
                'default_location': loc_pk
                ## Note - You do not have to fill out *all* fields
                })
            couch.uploadImage('lc/'+str(df.loc[i]['IPN'])+'.jpg')
            StockItem.create(api, { 'part': couch.pk, 'quantity': df.loc[i]['stock'], 'location': loc_pk, 'status':10 })
            SupplierPart.create(api,{
            'part':couch.pk,
            'supplier':company_pk,
            'SKU':df.loc[i]['IPN']
            })
            print('导入'+str(df.loc[i]['IPN'])+'成功')
        except:
            print('导入'+str(df.loc[i]['IPN'])+'失败')
        pbar.update(1/len(df.index))
    print('导入完毕')
    pbar.close()

In [13]:
url=input('url:')
username=input('username:')
password=getpass.getpass('password:')
api=login_inventree(url,username,password)

url:http://192.168.50.123:1337
username:3218
password:········
登录成功！


In [26]:
filepath=input('file:')
soup=read_lcsc_order(filepath)
df=deal_order_data(soup)
df

file:lcsc9.txt
文件读取成功！


,IPN,name,descripition,keyword,footprint,link,img,stock,price
0,C2835639,场效应管(MOSFET),N沟道 80V 170A,HY3708P,TO-220FB-3L,https://item.szlcsc.com/3000383.html,https://alimg.szlcsc.com/upload/public/product...,20,5.18
1,C86650,DC-DC电源芯片,降压型,LM2576T-12,TO-220-5,https://item.szlcsc.com/87827.html,https://alimg.szlcsc.com/upload/public/product...,10,5.52
2,C914529,直插独石电容(MLCC),1uF ±10% 100V,105K-100V-1812,"径向引线,P=5mm",https://item.szlcsc.com/985427.html,https://alimg.szlcsc.com/upload/public/product...,3,1.2338
3,C2905592,插件电感,1mH,PXT100577,"径向引线,6x8mm",https://item.szlcsc.com/3175858.html,https://static.szlcsc.com/ecp/public/static/im...,10,0.326847
4,C4649,散热板,散热片,15*10.5*21 H型散热片,15*10.5*21,https://item.szlcsc.com/5057.html,https://alimg.szlcsc.com/upload/public/product...,10,0.720783
5,C17622,贴片电阻,30Ω ±1% 1,0805W8F300JT5E,0805,https://item.szlcsc.com/18310.html,https://alimg.szlcsc.com/upload/public/product...,100,0.01536
6,C915156,功率电感,100uH ±10%,MSCD32-101KT,SMD,https://item.szlcsc.com/987381.html,https://alimg.szlcsc.com/upload/public/product...,20,0.164162
7,C2847237,引线型铝电解电容,100uF ±20% 80V,LZ101M0801013PE3,"径向引线,10x13mm",https://item.szlcsc.com/3043350.html,https://alimg.szlcsc.com/upload/public/product...,20,0.633266
8,C1713,贴片电容(MLCC),10uF ±10% 16V,CL21A106KOQNNNE,0805,https://item.szlcsc.com/2065.html,https://alimg.szlcsc.com/upload/public/product...,50,0.057115
9,C389518,发光二极管,普绿,MHT192CGCT,0603,https://item.szlcsc.com/364864.html,https://alimg.szlcsc.com/upload/public/product...,20,0.160273


In [18]:
cat_pk=Inventree_read_PartCategory(api)
company_pk=Inventree_read_company(api)
loc_pk=Inventree_read_location(api)

pk  name
1    电子元器件
19    lcsc
14    电子设备
13    直线电机零件
请输入插入的分类pk号：19
pk  name
3    东莞市雷宝电子有限公司
2    海美格磁石技术（深圳）有限公司
1    立创商城
请输入插入的供应商pk号：1
pk  name
1    电子元器件
8    1
9    2
10    货架1
11    货架2
请输入插入的默认仓库pk号：8


In [27]:
Inventree_Part_create(api,df,loc_pk,company_pk,cat_pk)

0.11764705882352941it [00:00,  1.35s/it]

导入C2835639失败
导入C86650失败


0.35294117647058826it [00:01,  4.32s/it]

导入C4649失败
导入C17622失败
导入C915156失败


0.5882352941176471it [00:02,  3.67s/it] 

导入C1713失败
导入C389518失败
导入C2985583失败


0.7058823529411765it [00:02,  2.84s/it]

导入C8465失败


1.0it [00:05,  5.66s/it]               

导入完成
